![alt text](w4_fig0.png "Title")

 Convert some Classic PlASM scripts from file CAD-LMPS_2008. 
 Free choice of number and type of scripts to Convert

In [1]:
 from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.0241791237106 seconds


c:\python27\lib\site-packages\larlib\larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


Definition of a column.

In [ ]:
def column(args):
    """ This function assembles a parametric column in pseudo-Corinthian style.
            :param dm: circonference diameter at the column basis
            :param h: column height
            :param hbase: hight of the column base
            :return: an object HPC"""

    dm, h, hbase = args
    cylinder = JOIN([TRUNCONE([dm/2, 0.8*dm/2, h])(24)])
    torus_bot = JOIN([TORUS([dm/3, dm/1.7])([16, 16])])
    torus_top = JOIN([TORUS([0.8*(dm/3), 0.8*(dm/1.7)])([16, 16])])
    base = T([1, 2])([7*dm/-12, 7*dm/-12])(CUBOID([7*dm/6, 7*dm/6, hbase]))
    base_top = T([1, 2])([7*dm/-12 , 7*dm/-12])(CUBOID([7*dm/6, 7*dm/6, dm/6]))
    capital = UNION([JOIN([TRUNCONE([0.8*dm/2, 1.2*dm/2, h/8])(4)]),
                     JOIN([R([1, 2])(PI/4)(TRUNCONE([0.8*dm/2, 1.2*dm/2 , h/8])(4))])])

    return TEXTURE("marmo.png")(TOP([base, TOP([torus_bot, TOP([cylinder, TOP([torus_top, TOP([capital, base_top])])])])]))


In [ ]:
# Model generated by instancing the Columna function
VIEW(column([0.5, 4.5, 0.3]))

![alt text](w4_fig1.png "Title")

Definition of an arch surface blending function.

In [ ]:
def archSurface(parameters):
    """this function generates the transfinite blending (linear Bezier interpolation) of two control curves:
       circumferences with same center and different radiuses rr and rr - w (for width), respectively.
            :param rr: radius of the extern circonference
            :param w: width of the arc
            :return: an HPC object"""

    rr, w, angle = parameters
    h = SIN(angle) * (rr * 2)
    c0 = BEZIERCURVE([[K(rr * COS(0))(2), 0.0, 0.0], [0.0, 0.0, K(h * SIN(1))(2)], [-K(rr * COS(0))(2), 0.0, 0.0]])
    c1 = BEZIERCURVE([[K((rr - w) * COS(0))(2), 0.0, 0.0], [0.0, 0.0, K((h - w) * SIN(1))(2)],[-K((rr - w) * COS(0))(2),
                                                                                               0.0, 0.0]])

    return BEZIER(S2)([c0, c1])

In [ ]:
VIEW(TEXTURE("marmo.png")(MAP(archSurface([1.0, 0.3, PI/6]))(EMBED(1)(PROD([Hpc(Grid([10*[1./10]])),  Hpc(Grid([10*[1./10]]))])))))

![alt text](w4_fig2.png "Title")

Definition of an arch.

In [ ]:
def ARCH(params):
    """ This function generates an arch
            :param length: the arc length
            :param w: the horizontal arc width
            :param depth: the arc depth
            :param angle: the  arc angle, usually given in the ancient architecture as a multiple of PI/6
            :return: an HPC object"""
    length, w, depth = params
    def ARCH0(angle):

        domain = EMBED(1)(PROD([Hpc(Grid([10 * [1. / 10]])), Hpc(Grid([10 * [1. / 10]]))]))
        rr = (length/2)
        h = SIN(angle)*(rr*2)

        c1 = BEZIERCURVE([[K(rr*COS(0))(2), 0.0, 0.0], [0.0, 0.0, -K(h*SIN(1))(2)], [-K(rr*COS(0))(2), 0.0, 0.0]])
        c2 = BEZIERCURVE([[K(rr*COS(0))(2), depth, 0.0],[0.0, depth, -K(h*SIN(1))(2)], [-K(rr*COS(0))(2), depth, 0.0]])
        c3 = BEZIERCURVE([[K((rr-w)*COS(0))(2), 0.0, 0.0],[0.0,0.0,K((h-w)*SIN(1))(2)], [-K((rr-w)*COS(0))(2), 0.0, 0.0]])
        c4 = BEZIERCURVE([[K((rr-w)*COS(0))(2),depth,0.0],[0.0,depth,K((h-w)*SIN(1))(2)],[-K((rr-w)*COS(0))(2), depth, 0.0]])

        out1 = MAP(archSurface([rr, w, angle]))(domain)
        out2 = T(2)(depth)(R([1, 2])(PI)(out1))
        out3 = R([1, 3])(PI)(MAP(BEZIER(S2)([c1, c2]))(domain))
        out4 = MAP(BEZIER(S2)([c3, c4]))(domain)

        return STRUCT([out1, out2, out3, out4])
    return ARCH0

In [ ]:
# singolar arch

VIEW(ARCH([2.0, 0.2, 0.3])(PI/6*2.5))


![alt text](w4_fig3.png "Title")

In [ ]:
# many arcs with different angles
VIEW(STRUCT([ARCH([2.0, 0.2, 0.3])(PI/6),  ARCH([2.0, 0.2, 0.3])(PI/6*1.5),  ARCH([2.0, 0.2, 0.3])(PI/6*2),
          ARCH([2.0, 0.2, 0.3])(PI/6*0.4),  ARCH([2.0, 0.2, 0.3])(PI/6*0.7),  ARCH([2.0, 0.2, 0.3])(PI/6*0.2)]))

![alt text](w4_fig4.png "Title")

Definition of half sphere

In [ ]:
def halfSphere(r):
    fx = K(r * -(SIN(1) * COS(0)))(1)
    fy = K(r * COS(0) * COS(1))(1)
    fz = K(r * SIN(0))(1)

    return [fx, fy, fz]

Definition of a truss generative function.

In [ ]:
def TRUSS(params1):
    """ This function defines a space truss of given length and height h
            :return: an HPC object"""

    length, h = params1
    def TRUSS0(params2):
        x, y, z = params2
        verts = [[-6, 0], [-3, 0], [-3, 2], [0, 0], [0, 4], [3, 0], [3, 2], [6, 0]]
        cells = [[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [4, 7], [5, 7], [6, 7], [6, 8], [7, 8]]
        pols = AA(LIST)(range(1, 14))

        print pols

        return OFFSET([x, y, z])(S([1, 3])([length/12, h/4])(MKPOL([verts, cells, pols])))
    return TRUSS0

In [ ]:
VIEW(TRUSS([24.0, 4.0])([0.3, 0.3, 0.3]))

![alt text](w4_fig5.png "Title")

![alt text](w4_fig6.png "Title")